In [16]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: azml-workspace
Azure region: westeurope
Subscription id: 0693b1d2-906d-43da-8982-fe9e50bb367c
Resource group: awrgstudak


In [17]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "opti-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [18]:
print("SDK version:", azureml.core.VERSION)

SDK version: 1.33.0


In [19]:
from azureml.core import Environment

sklearn_env = Environment.get(ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
        

In [20]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import Environment
from azureml.core import ScriptRunConfig
import os


# Specify parameter sampler

ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = SKLearn(source_directory = "./",compute_target=compute_target,vm_size='STANDARD_D2_V2', entry_script="train.py")

#'SKLearn' estimator is deprecated.  'ScriptRunConfig' from 'azureml.core.script_run_config' therefore was used
src = ScriptRunConfig(source_directory="./",
                      script="train.py",
                      compute_target=compute_target,
                      environment=sklearn_env)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, 
                                     hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)

In [21]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)
RunDetails(hyperdrive_run).show()

RunId: HD_b1e5071c-700d-4515-8638-19dceb81f2a5
Web View: https://ml.azure.com/runs/HD_b1e5071c-700d-4515-8638-19dceb81f2a5?wsid=/subscriptions/0693b1d2-906d-43da-8982-fe9e50bb367c/resourcegroups/awrgstudak/workspaces/azml-workspace&tid=49707145-68cd-42ed-bde7-e744bd74c8eb

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-01-23T10:37:13.543729][API][INFO]Experiment created<END>\n""<START>[2022-01-23T10:37:14.706877][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-01-23T10:37:15.383683][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_b1e5071c-700d-4515-8638-19dceb81f2a5
Web View: https://ml.azure.com/runs/HD_b1e5071c-700d-4515-8638-19dceb81f2a5?wsid=/subscriptions/0693b1d2-906d-43da-8982-fe9e50bb367c/resourcegroups/awrgstudak/workspaces/azml-workspace&tid=49707145-68cd-42ed-bde7-e744bd74c8eb



_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [22]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = hyperdrive_run.get_metrics()
best_run_files=best_run.get_file_names()
print(best_run.id)
print(best_run_files)
print(best_run_metrics)
hyperdrive_model=best_run.register_model(model_name='best-hyperdrive-model', model_path='outputs/model.joblib')

HD_b1e5071c-700d-4515-8638-19dceb81f2a5_10
['logs/azureml/21_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']
{'HD_b1e5071c-700d-4515-8638-19dceb81f2a5_9': {'Regularization Strength:': 1000.0, 'Max iterations:': 200, 'Accuracy': 0.910980320029428}, 'HD_b1e5071c-700d-4515-8638-19dceb81f2a5_11': {'Regularization Strength:': 0.1, 'Max iterations:': 200, 'Accuracy': 0.910152657715652}, 'HD_b1e5071c-700d-4515-8638-19dceb81f2a5_8': {'Regularization Strength:': 0.1, 'Max iterations:': 50, 'Accuracy': 0.9115320949052786}, 'HD_b1e5071c-700d-4515-8638-19dceb81f2a5_10': {'Regularization Strength:': 1.0, 'Max

In [23]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url= 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path=url,validate='False',separator=',',infer_column_types=True,include_path=False,
set_column_types=None,support_multi_line=False,partition_format=None)

In [24]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [25]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=4)

In [26]:
# Submit your automl run

### YOUR CODE HERE ###
experiment = Experiment(ws, "automl-experiment")
automl_run= experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_7e685d9f-4701-4856-aeba-bed47c6de138,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
****************************************************************************************************


In [27]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()

In [28]:
best_run.get_metrics()

{'log_loss': 0.20927722477235683,
 'recall_score_micro': 0.9178147859761392,
 'f1_score_weighted': 0.914772997823162,
 'weighted_accuracy': 0.9570770776930784,
 'matthews_correlation': 0.559778164431423,
 'accuracy': 0.9178147859761392,
 'precision_score_weighted': 0.9129277286112691,
 'f1_score_micro': 0.9178147859761392,
 'average_precision_score_weighted': 0.9562175058811925,
 'precision_score_macro': 0.8017074346325898,
 'f1_score_macro': 0.7783809355933454,
 'recall_score_macro': 0.7597509995723899,
 'average_precision_score_macro': 0.8277237226306381,
 'average_precision_score_micro': 0.981892843694737,
 'balanced_accuracy': 0.7597509995723899,
 'norm_macro_recall': 0.5195019991447798,
 'recall_score_weighted': 0.9178147859761392,
 'precision_score_micro': 0.9178147859761392,
 'AUC_weighted': 0.9487359396898007,
 'AUC_macro': 0.9487359396898007,
 'AUC_micro': 0.9811311037228926,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_7e685d9f-4701-4856-aeba-bed47c6de138_29

In [29]:
fitted_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
)), ('logisticregression', LogisticRegression(C=51.79474679231202, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('13', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('sgdclassifierwrapper', SGDClassifierWrapper(alpha=7.5510448979591835, class_weight='balanced', eta0=0.001, fit_intercept=True, l1_ratio=0.42857142857142855

In [30]:
best_run
best_automl_model= best_run.register_model(model_name='best-automl-model', model_path='outputs/model.pkl')

In [31]:
compute_target.delete()